<a href="https://colab.research.google.com/github/0ShNa0/DatasetNN/blob/main/NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json


In [2]:
from sys import modules
import re
compile1=[]
with open('output_data.json') as json_file:
  data=json.load(json_file)
  for d in data:
    for block in d['ai-generated']:
      if(block['category']==5):
        sent=block['sentence'].lstrip()
        pattern = re.compile(r'(\d+),(\d+)')
        mod_sent=pattern.sub(r'\1\2', sent)
        if(mod_sent[0]>='0' and mod_sent[0]<='9' and mod_sent[1]=='.'):
          compile1.append(mod_sent[2:])
        else:
          compile1.append(mod_sent)

In [3]:
import re
process_data=dict()
for sentence in compile1:
  numbers=re.findall('[0-9]+', sentence)
  process_data[sentence]=numbers



In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")

model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

def paraphrase(
    question,
    num_beams=10,
    num_beam_groups=10,
    num_return_sequences=10,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids

    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
process_data2={}
for keys,_ in process_data.items():
  process_data2[keys]=paraphrase(keys)



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [7]:
csv_file_name = 'process_data2NN.csv'
import csv
# Writing the dictionary to a CSV file
with open(csv_file_name, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # Write the header
    csv_writer.writerow(process_data2.keys())

    # Write the data
    csv_writer.writerows(zip(*process_data2.values()))

In [10]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch

def check_entailment(query, text):
  tokenizer = RobertaTokenizer.from_pretrained('roberta-large-mnli')
  model = RobertaForSequenceClassification.from_pretrained('roberta-large-mnli')
  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  inputs = tokenizer.encode_plus(query, text, add_special_tokens=True, padding='longest', truncation=True, return_tensors='pt')
  input_ids = inputs['input_ids']
  attention_mask = inputs['attention_mask']
  with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    entailment_score = probabilities[:, 1].item()

  return entailment_score


In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
scorelist=dict()
for key,rephrase in process_data2.items():
  scorelist[key]=[check_entailment(key,sentence) for sentence in rephrase]


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another tas

In [13]:
for _,values in scorelist.items():
  print(values)

[0.03590904176235199, 0.004733892623335123, 0.044012054800987244, 0.009596575982868671, 0.004853073973208666, 0.5167971253395081, 0.004742083139717579, 0.005773508921265602, 0.004702579230070114, 0.005035745911300182]
[0.006229540333151817, 0.004579564556479454, 0.30218440294265747, 0.009598249569535255, 0.19463759660720825, 0.3065827786922455, 0.0109929908066988, 0.9169411659240723, 0.005546960514038801, 0.01317534688860178]
[0.004498389083892107, 0.004494279157370329, 0.004480576142668724, 0.00927585270255804, 0.004306063987314701, 0.004803628195077181, 0.004560932982712984, 0.036622319370508194, 0.030543847009539604, 0.7395308017730713]
[0.48884257674217224, 0.9543564319610596, 0.5710175633430481, 0.6285434365272522, 0.7760863304138184, 0.9927091598510742, 0.32427918910980225, 0.3492506444454193, 0.4861668646335602, 0.6556385159492493]
[0.004261363763362169, 0.004606989677995443, 0.013540523126721382, 0.08122137933969498, 0.004193081054836512, 0.2047627717256546, 0.00723713636398315

In [14]:
csv_file_name = 'output.csv'
import csv
# Writing the dictionary to a CSV file
with open(csv_file_name, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # Write the header
    csv_writer.writerow(scorelist.keys())

    # Write the data
    csv_writer.writerows(zip(*scorelist.values()))

In [15]:

from google.colab import files
files.download('output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
import csv

def csv_to_dict(file_path):
    result_dict = {}
    with open(file_path, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            for key, value in row.items():
                result_dict.setdefault(key, []).append(value)
    return result_dict

# Example usage
csv_file_path = 'output.csv'
result_dict = csv_to_dict(csv_file_path)

print(result_dict)


{'Chinese cleavers have wide and flat blades, usually measuring at least six inches long.': ['0.03590904176235199', '0.004733892623335123', '0.044012054800987244', '0.009596575982868671', '0.004853073973208666', '0.5167971253395081', '0.004742083139717579', '0.005773508921265602', '0.004702579230070114', '0.005035745911300182'], 'Hamilton Beach FlexBrew Single Serve Coffee Maker: This machine is perfect for those who want to make one cup of coffee at a time.': ['0.006229540333151817', '0.004579564556479454', '0.30218440294265747', '0.009598249569535255', '0.19463759660720825', '0.3065827786922455', '0.0109929908066988', '0.9169411659240723', '0.005546960514038801', '0.01317534688860178'], 'He completed 26-of-38 passes for 286 yards and two touchdowns, while also scrambling for a team-high 29 yards on four carries.': ['0.004498389083892107', '0.004494279157370329', '0.004480576142668724', '0.00927585270255804', '0.004306063987314701', '0.004803628195077181', '0.004560932982712984', '0.0

In [11]:
allscores=[]
for _,values in result_dict.items():
  for val in values:
    allscores.append(float(val))
print(allscores)

import numpy as np

sorted_scores = np.sort(allscores)


median = np.median(sorted_scores)

[0.03590904176235199, 0.004733892623335123, 0.044012054800987244, 0.009596575982868671, 0.004853073973208666, 0.5167971253395081, 0.004742083139717579, 0.005773508921265602, 0.004702579230070114, 0.005035745911300182, 0.006229540333151817, 0.004579564556479454, 0.30218440294265747, 0.009598249569535255, 0.19463759660720825, 0.3065827786922455, 0.0109929908066988, 0.9169411659240723, 0.005546960514038801, 0.01317534688860178, 0.004498389083892107, 0.004494279157370329, 0.004480576142668724, 0.00927585270255804, 0.004306063987314701, 0.004803628195077181, 0.004560932982712984, 0.036622319370508194, 0.030543847009539604, 0.7395308017730713, 0.48884257674217224, 0.9543564319610596, 0.5710175633430481, 0.6285434365272522, 0.7760863304138184, 0.9927091598510742, 0.32427918910980225, 0.3492506444454193, 0.4861668646335602, 0.6556385159492493, 0.004261363763362169, 0.004606989677995443, 0.013540523126721382, 0.08122137933969498, 0.004193081054836512, 0.2047627717256546, 0.007237136363983154, 0

In [12]:
print(median)
percentile_75=np.percentile(sorted_scores,75)
count_above_75th_percentile = sum(score >= percentile_75 for score in sorted_scores)
print(percentile_75)
print(count_above_75th_percentile)

0.0067568933591246605
0.03693147096782923
235


In [8]:

from google.colab import files
files.download('process_data2NN.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
final_data=[]
for key,values1,values2 in zip(result_dict.keys(),process_data2.values(),result_dict.values()):
  for sent,score in zip(values1,values2):
    if(float(score)>=median and len(key.strip())>0):
      final_data.append([key,sent])

In [33]:
for data in final_data:
  print(data)

['Chinese cleavers have wide and flat blades, usually measuring at least six inches long.', 'Chinese cleavers are known for their flat, wide blades that are typically at least six inches long.']
['Chinese cleavers have wide and flat blades, usually measuring at least six inches long.', 'Western-thick blades with a minimum length of six inches are characteristic of Chinese cleavers, which are typically wide and flat.']
['Chinese cleavers have wide and flat blades, usually measuring at least six inches long.', 'At least six inches in length is the standard measurement for Chinese cleaver blades, which are flat and wide.']
['Chinese cleavers have wide and flat blades, usually measuring at least six inches long.', 'With at least six inches in length, Chinese cleavers are known for their flat and wide blades that are typically wider.']
['Hamilton Beach FlexBrew Single Serve Coffee Maker: This machine is perfect for those who want to make one cup of coffee at a time.', 'Those seeking instant

In [34]:
import pandas as pd
df = pd.DataFrame((final_data), columns=["Column1", "Column2"])
print(df)

                                               Column1  \
0    Chinese cleavers have wide and flat blades, us...   
1    Chinese cleavers have wide and flat blades, us...   
2    Chinese cleavers have wide and flat blades, us...   
3    Chinese cleavers have wide and flat blades, us...   
4    Hamilton Beach FlexBrew Single Serve Coffee Ma...   
..                                                 ...   
455  Without a steady income, Susi had to dip into ...   
456  Without a steady income, Susi had to dip into ...   
457  Without a steady income, Susi had to dip into ...   
458  Without a steady income, Susi had to dip into ...   
459  Without a steady income, Susi had to dip into ...   

                                               Column2  
0    Chinese cleavers are known for their flat, wid...  
1    Western-thick blades with a minimum length of ...  
2    At least six inches in length is the standard ...  
3    With at least six inches in length, Chinese cl...  
4    Those seeking

In [35]:
import csv

custom_header = ["Original","Paraphrases"]
df.columns = custom_header


df.to_csv("final_data.csv", index=True)

In [36]:

from google.colab import files
files.download('final_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>